The goal of this notebook is to outline a small modification of the expectation maximization (EM hereafter) algorithm to handle partially labeled data. We begin by reviewing the standard EM algorithm and then describing the modification in the next block.

# Standard EM

Recall that the standard EM model is as follows. We have $m$ unlabeled data points $\{ x^{(i)} \}_{i=1}^m$ inside $\mathbb{R}^d$ and we assume they are drawn independently from an unknown distribution $p(x;\theta)$ parametrized by $\theta \in \mathbb{R^p}$. We want to choose $\theta$ to maximize the joint log likelihood,

$$\ell(\theta) = \sum_{i=1}^m \log p(x^{(i)}; \theta).$$

We assume additionally the existence of $m$ 'hidden' or latent random variables $z^{(i)}$ each taking values in a discrete (possibly different) set of values $\{\hat{z}^{(i)}_j\}_{j=1}^{n_i}$. Assume that $Q^{(i)}$ is the distribution of $z^{(i)}$, so that 

$$Q^{(i)}(\hat{z}^{(i)}) = p(z^{(i)} = \hat{z}^{(i)}).$$

We can then write the log likelihood in terms of the joint density of $x^{(i)}$ and $z^{(i)}$ as follows:

$$\ell(\theta) = \sum_{i=1}^m \log \sum_{j=1}^{n_i} p(x^{(i)}, \hat{z}^{(i)}_j ) = \sum_{i=1}^m \log \left( \sum_{j=1}^{n_i} Q^{(i)}(\hat{z}^{(i)}_j) \frac{p(x^{(i)}, \hat{z}^{(i)}_j )}{Q^{(i)}(\hat{z}^{(i)}_j)}\right).$$

By Jensen's inequality, we then have 

$$ \ell(\theta) \geq \sum_{i=1}^m \sum_{j=1}^{n_i} Q^{(i)}(\hat{z}^{(i)}_j) \log \frac{p(x^{(i)}, \hat{z}^{(i)}_j ; \theta)}{Q^{(i)}(\hat{z}^{(i)}_j)} = \tilde{\ell}(\theta,Q).$$

This is true for any choices of the distributions $Q^{(i)}$, but for fixed $\theta$, the right hand side is maximized when equality holds, i.e. for the choice 

$$Q_\theta^{(i)}(\hat{z}^{(i)}_j) = p(z^{(i)} = \hat{z}^{(i)}_j \, | \, x^{(i)};\theta).$$

The algorithm now proceeds in two steps. First, starting from a given value of $\theta$, $\theta_0$, we set $Q_{\theta_0}^{(i)}(\hat{z}^{(i)}_j) = p(z^{(i)} = \hat{z}^{(i)}_j \, | \, x^{(i)};\theta_0)$ and then we set 

$$\theta_1 = \argmax_\theta \tilde{\ell}(\theta, Q_{\theta_0}).$$

It follows that $\ell(\theta_1) \geq \ell(\theta_0)$, since

$$\ell(\theta_1) \geq \tilde{\ell}(\theta_1,Q_{\theta_0}) \geq \tilde{\ell}(\theta_0,Q_{\theta_0}) = \ell(\theta_0)$$

where the last inequality follows from the fact that $\ell(\theta) = \tilde{\ell}(\theta,Q_\theta)$ for any $\theta$.


# Semi-supervised EM

Now we consider the same starting set-up of $m$ unlabeled data points $\{ x^{(i)} \}_{i=1}^m$ but we assume that we also have $\tilde{m}$ additional *labeled* data points, $\{(\tilde{x}^{(k)} ,\tilde{z}^{(k)} )\}_{k=1}^{\tilde{m}}$. We want to simultaneously fit our model to the unlabeled data and the labeled data. If we were only concerned with the labeled data, we would attempt to maximize the joint likelihood,

$$\ell_{s}(\theta) = \sum_{k=1}^{\tilde{m}} \log p(\tilde{x}^{(k)} , \tilde{z}^{(k)} ; \theta).$$

Instead, we will optimize for both the labeled and unlabeled data by maximizing a weighted sum of the two objectives, 

$$L(\theta) = \sum_{i=1}^m \log \sum_{j=1}^{n_i} p(x^{(i)}, \hat{z}^{(i)}_j ) + \alpha \sum_{k=1}^{\tilde{m}} \log p(\tilde{x}^{(k)} , \tilde{z}^{(k)} ; \theta)$$

With $\tilde{\ell}(\theta,Q)$ defined the same way as in the previous block, again we have that for any $Q$,

$$L(\theta) \geq \tilde{\ell}(\theta,Q) + \alpha \ell_s(\theta)$$

and when $Q = Q_\theta$ (same definition as above), the above holds with equality. Now starting from $\theta_0$, we set $Q = Q_{\theta_0}$ and $\theta_1 = \argmax_\theta \tilde{\ell}(\theta,Q_{\theta_0}) + \alpha \ell_s(\theta)$. Then

$$L(\theta_1) \geq \tilde{\ell}(\theta_1,Q_{\theta_0}) + \alpha \ell_s(\theta_1) \geq \tilde{\ell}(\theta_0,Q_{\theta_0}) + \alpha \ell_s(\theta_0) = L(\theta_0).$$

Therefore we only need to modify the 'M' step above. The new algorithm updates $\theta_t \mapsto \theta_{t+1}$ as follows:

1. Set $Q_{\theta_t}^{(i)}(\hat{z}^{(i)}_j) = p(z^{(i)} = \hat{z}^{(i)}_j \, | \, x^{(i)} ; \theta_t)$
2. Set $\theta_{t+1} = \argmax_\theta \tilde{\ell}(\theta, Q_{\theta_t}) + \alpha \ell_s(\theta)$

# Semi-supervised mixed Gaussians model

Now we wish to apply this framework to the mixture of Gaussians model. That is, we assume that each sample $x^{(i)}$ is drawn from one of $n$ distinct Gaussians and we let $z^{(i)}$ be the multinomially distributed random variable indicating which Gaussian $x^{(i)}$ was drawn from. This means that $n_i = N$ for each $i \in \{1 , \dots , m\}$, and $\hat{z}^{(i)}_j = j$, $j \in \{1 , \dots , N\}$. The labels $\tilde{z}^{(i)}$ indicate which Gaussian distribution $\tilde{x}^{(i)}$ was drawn from.

The model is parametrized by the means and covariance matrices of each of the $N$ Gaussians, 

$$\mu = (\mu_1 , \dots , \mu_N) \in \mathbb{R}^{N \times d}$$
$$\Sigma = (\Sigma_1 , \dots , \Sigma_N) \in \mathbb{R}^{N \times d \times d}$$

as well as the multinomial probabilities,

$$\phi = (\phi_1 , \dots , \phi_N) \in \mathbb{R}^N$$

where $\phi_j = p(z^{(i)} = j)$. That is, we have 

$$p(x^{(i)} \, | \, z^{(i)} = j ; \theta) = [(2\pi)^n|\Sigma_j|]^{-\frac{1}{2}} \exp \left(-\frac{1}{2} \langle \Sigma_j^{-1}(x^{(i)} - \mu_j) , (x^{(i)} - \mu_j) \rangle \right)$$


In the 'E' step, we set 

$$Q^{(i)}(j) = p(z^{(i)} = j \, | \, x^{(i)}; \theta) = \frac{p(x^{(i)} \, | \, z^{(i)} = j ; \theta) p(z^{(i)} = j)}{p(x^{(i)};\theta)}$$

or 

$$Q^{(i)}(j) = \frac{p(x^{(i)} \, | \, z^{(i)} = j ; \theta)\phi_j}{\sum_{k=1}^N p(x^{(i)} \, | \, z^{(i)} = k ; \theta)\phi_k}$$

Now we write down the update rules for the parameters from the 'M' step. The objective function to be maximized is

$$J(\theta) = \tilde{\ell}(\theta, Q) + \alpha \ell_s(\theta) = \sum_{i=1}^m \sum_{j=1}^N Q^{(i)}(j) \log \frac{p(x^{(i)}, z^{(i)} = j ; \theta)}{Q^{(i)}(j)} + \alpha \sum_{k=1}^{\tilde{m}} \log p(\tilde{x}^{(k)}, \tilde{z}^{(k)}; \theta)$$

for notational convenience we will let 

$$\tilde{m}_j = |\{k \, | \, \tilde{z}^{(k)} = j\}|$$

be the number of labeled examples with label '$j$'. Thus, $\sum_{j=1}^N \tilde{m}_j = \tilde{m}$.



# The $\phi$ update rules

If we drop all of the terms which don't depend on $\phi$ from $J$, we have

$$ \nabla_\phi J(\theta) = \nabla_\phi \left( \sum_{i=1}^{m} Q^{(i)}(j) \log \phi_j + \alpha \sum_{k=1}^{\tilde{m}} \log \phi_{\tilde{z}^{(k)}}\right) $$

Since we need to maximize $J$ subject to the constraint $\sum \phi_j  - 1 = 0$, we are looking for points such that 

$$\nabla_\phi J(\theta) = (\lambda , \dots , \lambda)$$

for some $\lambda \in \mathbb{R}$. Writing this in components, we have for each $a \in \{1,\dots,N\}$,

$$\frac{\partial J}{\partial \phi_a} = \sum_{i=1}^m \frac{Q^{(i)}(a)}{\phi_a} + \alpha \frac{\tilde{m}_a}{\phi_a} = \lambda $$

which is equivalent to 

$$\phi_a = \frac{1}{\lambda} \sum_{i=1}^m Q^{(i)}(a) + \alpha \tilde{m}_a$$

and if we sum this equation over $a$, the right hand side must be 1, which means that $\lambda = m + \alpha\tilde{m}$.

# The $\mu$ update rules

Similarly ignoring all the terms that don't depend on $\mu$, we have, for each $a \in \{1,\dots, N\}$

$$\nabla_{\mu_a} J = \nabla_{\mu_a} \left( \sum_{i=1}^m \sum_{j=1}^N -\frac{1}{2} Q^{(i)}(j) \bigg\langle \Sigma^{-1}_j (x^{(i)} - \mu_j) , (x^{(i)} - \mu_j) \bigg\rangle + \alpha \sum_{k=1}^{\tilde{m}} -\frac{1}{2} \bigg\langle \Sigma_{\mu_{\tilde{z}^{(k)}}} (\tilde{x}^{(k)} - \mu_{\tilde{z}^{(k)}}), (\tilde{x}^{(k)} - \mu_{\tilde{z}^{(k)}}) \bigg\rangle    \right)$$


Since 

$$\nabla_v \left(v \mapsto \frac{1}{2}\langle Av , v \rangle\right) = Av$$

when $A$ is a symmetric matrix, we get

$$\nabla_{\mu_a} J = \sum_{i=1}^m Q^{(i)}(a) \Sigma^{-1}_{a} (x^{(i)} - \mu_a) + \alpha \sum_{k=1}^{\tilde{m}} \Sigma^{-1}_a (\tilde{x}^{(k)} - \mu_a) 1\{ \tilde{z}^{(k)} = a \}$$

and since $\Sigma_a$ is invertible, this equals zero exactly when 

$$\sum_{i=1}^m Q^{(i)}(a) (x^{(i)} - \mu_a) + \alpha \sum_{k=1}^{\tilde{m}} (\tilde{x}^{(k)} - \mu_a) 1\{ \tilde{z}^{(k)} = a \} = 0$$

Rearranging this, we get in the end,

$$\mu_a = \frac{1}{\left( \sum_{i=1}^m Q^{(i)}(a) + \alpha\tilde{m}_a \right)}\left(\sum_{i=1}^m Q^{(i)}(a) x^{(i)} + \alpha \sum_{k=1}^{\tilde{m}} \tilde{x}^{(k)} 1\{z^{(k)} = a\} \right). $$

# The $\Sigma$ update rules

As before we will calculate the gradient with respect to the matrix $\Sigma_a$.

$$\nabla_{\Sigma_a} J = \nabla_{\Sigma_a} \tilde{\ell}(\cdot, Q) + \alpha \nabla_{\Sigma_a} \ell_s $$

$$\nabla_{\Sigma_a} \tilde{\ell}(\cdot,Q) = \nabla_{\Sigma_a} -\frac{1}{2}\left( \sum_{i=1}^m \sum_{j=1}^N Q^{(i)}(j) \left( \log\det \Sigma_j + \bigg \langle \Sigma_j^{-1}(x^{(i)} - \mu_j) , (x^{(i)} - \mu_j) \bigg\rangle \right) \right)$$

$$\nabla_{\Sigma_a} \ell_s = \nabla_{\Sigma_a} \left( \sum_{k=1}^N -\frac{1}{2}\log\det \Sigma_{\tilde{z}^{(k)}} -\frac{1}{2} \bigg \langle \Sigma_{\tilde{z}^{(k)}}^{-1}(\tilde{x}^{(k)} - \mu_{\tilde{z}^{(k)}}) , (\tilde{x}^{(k)} - \mu_{\tilde{z}^{(k)}}) \bigg\rangle \right)$$

As with the previous update rules, both cases involve almost the same calculation, so we write out the first case and then state the end result.

We use the formulas

$$d_A(\log \det A)(B) = \text{tr}\,(A^{-1}B)$$

$$d_A(\langle Av , v\rangle)(B) = \text{tr}\, (vv^T B)$$

$$d_A(A^{-1})(B) = -A^{-1}BA^{-1}$$

Then, taking the derivative in the direction $B$ of $\tilde{\ell}(\cdot,Q)$ we get

$$ d_{\Sigma_a}\tilde{\ell}(\cdot,Q)(B) = -\frac{1}{2}\sum_{i=1}^m Q^{(i)}_a \left(\text{tr} \, (\Sigma_a^{-1} B) - \text{tr} \, \left( (x^{(i)} - \mu_a)(x^{(i)} - \mu_a)^T \Sigma_a^{-1}B\Sigma_a^{-1} \right)\right)$$

and in the end, adding in the contribution from $\ell_s$, we find that

$$\Sigma_a = \frac{1}{\left(\sum_{i=1}^m Q^{(i)}(a) + \alpha\tilde{m}_a \right)}\left( \sum_{i=1}^m Q^{(i)}(a)(x^{(i)} - \mu_a)(x^{(i)} - \mu_a)^T + \alpha\sum_{k=1}^{\tilde{m}} (\tilde{x}^{(k)} - \mu_a)(\tilde{x}^{(k)} - \mu_a)^T 1\{\tilde{z}^{(k)} = a\}\right)$$ 


# Notes on coding unsupervised GMM

The code in 'p04_gmm.py' fits an unsupervised GMM to the data found in .\data\ds3_training. The dataset is loaded as a numpy array of shape (1000,2) called 'x' (i.e. the data points lie in $\mathbb{R}^d$, $d = 2$).

There are 4 Gaussians from which the data is drawn (K = 4). The goal is to create a 4-color scatter plot of the data, with the color of each datapoint indicating the Gaussian it has the highest probability of belonging to. In order to fit the model, we run the EM algorithm to maximize the log-likelihood of the data. We loop the E-M steps until the increase in log likelihood falls below a threshold amount (eps = 1e-3 in the code).

## Initializing the parameters

The parameters in the problem are:

1. The probabilities $\phi_j = p(z^{(i) = j})$ 
2. The means and covariance matrices of the K gaussians, $\mu_j$, $\Sigma_j$.

These are stored as $\phi$ a (K,) shape array, $\mu$, a (K,d) shape array, and $\Sigma$, a (K,d,d) shape array.

The weight matrix w in this problem is shape (m,K); row i contains the probabilities that data point i belongs to each of the K possible Gaussians. To make predicted labels, we simply take the argmax of w along its rows to obtain the index of the Gaussian that $x^{(i)}$ was drawn from with highest probability. Then we plot the data and color it accordingly to these labels.

We initialize $phi$ and $w$ to put equal probability (1/K) on every Gaussian. To initialize the means and covariances, we split the data uniformly at random into K m/K size subsets and calculate the sample means and covariances for each. The helper function `split_inputs`

1. Chooses a permutation of [0,...,m-1] at random and applies this permutation to the rows of x.
2. splits the indicies [0,m-1] into K equal sequential blocks and sets the ith sample to be the ith block of m/K rows of the shuffled array x.

** Note: np.random.shuffle does not produce a copy of the original array, so we have to copy x first. It is also only applied to the first axis, which is exactly what we want here.

## Updating the weight matrix

The weight matrix is defined by

$$w_{ij} = \frac{p(x^{(i)} \, | \, z^{(i)} = j ; \theta)\phi_j}{\sum_{k=1}^N p(x^{(i)} \, | \, z^{(i)} = k ; \theta)\phi_k}$$

We define the helper function `cond_probs` to take in the dataset $x$ and parameters $\mu$ and $\Sigma$ and output the (m,K) shape array

$$P_{ij} = p(x^{(i)} \, | \, z^{(i)} = j ; \theta)$$

Then, $w$ is just $w_{ij} = P_{ij}\phi_j / (P\phi)_i$.

## The helper functions

### `outer_prods`

There is a 4-tensor which naturally comes up repeatedly in the update formulas. It is:

$$A_{ijkl} = [(x^{(i)} - \mu^{(j)}) \otimes (x^{(i)} - \mu^{(j)})]_{kl}$$

This is easily encoded using broadcasting. First, the 3-tensor $B_{ijk} = (x^{(i)} - \mu^{(j)})_k$ is obtained by reshaping $x$ to be (m,1,d) and $\mu$ to be (1,K,d). Then $B = x - \mu$. To take the outer square wrt the last axis, reshape B to (m,K,d,1) and (m,K,1,d) and multiply.

### `cond_probs`

Note that we are trying to code the matrix

$$P_{ij} = (2\pi)^{-d/2}|\Sigma_j|^{-1/2} \exp \left(-\frac{1}{2} \Sigma_{jkl} (x^{(i)} - \mu^{(j)})_k (x^{(i)} - \mu^{(j)})_l \right)$$

The core issue is how to code the argument of the exponential, but in terms of the tensor $A$ returned by `outer_prods`, this is just

(sigma.reshape((1,K,d,d)) * A_{ijkl} ).sum(axis = (2,3)) 

Here, we use the fact that we can sum over multiple axes at once by listing them as a tuple.


# Notes on coding semi-supervised GMM

In the SS setting, updating the weight matrix from the current state of parameters is exactly the same. Therefore we should write an `update_w` helper function so as not to repeat code.

### Labeled examples

1. Create a (K,) vector whose entries are the number of labeled examples with z = j, $\tilde{m_j}$ in the previous blocks.

As of 5.8 evening, it runs but does not produce correct results. something is wrong mathematically probably. Be careful: it is better not to take
log after exp of a very small negative number, can be interpreted as log of zero. The way I wrote the function which computes the values
p(x^{(i)} | z^{(i)} = j; params), I need to take log of this to compute the supervised log likelihood.

In [1]:
import numpy as np

x = np.arange(4).reshape((2,2))
y = np.zeros((1,1,1))
y[0] = x
print(y)

ValueError: could not broadcast input array from shape (2,2) into shape (1,1)

In [20]:


z = np.array([2,1,3,1,2,1,1,3,3,2]) - 1
phi = np.array([1/2,1/3,1/9])

square_phi = np.ones((10,1)) * phi.reshape(1,-1)

print(square_phi[range(10),z])



[0.33333333 0.5        0.11111111 0.5        0.33333333 0.5
 0.5        0.11111111 0.11111111 0.33333333]
